In [27]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

from IPython.display import display, Markdown

In [9]:
# Load environment variables from .env file
load_dotenv()

True

In [ ]:
# %pip install langchain langchain-openai langchain-groq langchain-community langchain-chroma

In [79]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.memory import ChatMessageHistory
from langchain_core.runnables import RunnablePassthrough
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from pydantic import BaseModel, Field


In [15]:
llm_model = os.environ["OPENAI_MODEL"]
# llm_model = "moonshotai/kimi-k2-instruct"
print(llm_model)
llm = ChatOpenAI(model=llm_model, temperature=0.1)
# model = ChatGroq(model=llm_model, temperature=0.1)

response = llm.invoke("Tell me a joke about data scientists")
print(response.content)

gpt-4o-mini
Why did the data scientist break up with the statistician?

Because she found him too mean!


In [16]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant. Answer all questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm # Chain

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)
print(ai_msg.content)

I translated "I love programming" into French as "J'adore la programmation."


In [28]:
SYSTEM_PROMPT = """You are an experienced software architect that assists a novice developer 
to design a system. """

qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", SYSTEM_PROMPT),
            ("human", "{question}"),
        ]
)

query = "What are the prons and cons of the Proxy design pattern?"

qa_chain = qa_prompt | llm

result = qa_chain.invoke(input=query)
# print(result.content)
display(Markdown(result.content))


The Proxy design pattern is a structural design pattern that provides an object representing another object. It acts as an intermediary, controlling access to the original object. Here are the pros and cons of using the Proxy design pattern:

### Pros:

1. **Control Access**: Proxies can control access to the real object, allowing for additional security measures or access restrictions.

2. **Lazy Initialization**: Proxies can delay the creation of expensive objects until they are actually needed, which can improve performance and resource management.

3. **Remote Access**: In distributed systems, proxies can facilitate communication with remote objects, making it easier to work with remote services as if they were local.

4. **Logging and Monitoring**: Proxies can be used to log requests and responses, providing a way to monitor interactions with the real object for debugging or auditing purposes.

5. **Decoupling**: Proxies can decouple the client from the real object, allowing for easier changes to the real object without affecting the client.

6. **Additional Functionality**: Proxies can add additional functionality (like caching, validation, etc.) without modifying the original object.

### Cons:

1. **Increased Complexity**: Introducing proxies can add complexity to the system, making it harder to understand and maintain.

2. **Performance Overhead**: While proxies can improve performance in some cases (like lazy loading), they can also introduce overhead due to the additional layer of indirection, especially if not implemented efficiently.

3. **Potential for Misuse**: If not designed carefully, proxies can lead to misuse or overuse, where they are used in situations where they are not necessary, complicating the architecture.

4. **Debugging Difficulty**: The additional layer can make debugging more challenging, as it may not be immediately clear whether an issue lies with the proxy or the real object.

5. **Tight Coupling with Proxy Logic**: If the client code is tightly coupled with the proxy's behavior, it may become difficult to switch to a different implementation or remove the proxy altogether.

6. **Limited Functionality**: Proxies may not support all the functionalities of the real object, especially if they are not designed to forward all method calls properly.

### Conclusion

The Proxy design pattern can be a powerful tool in software design, providing benefits such as access control, lazy initialization, and additional functionality. However, it also introduces complexity and potential performance overhead. It's essential to carefully consider the specific use case and requirements of your system before implementing this pattern.

In [31]:
CONTEXTUALIZED_PROMPT = """Given a chat history and the latest developer's question
    which might reference context in the chat history, formulate a standalone question
    that can be understood without the chat history. Do NOT answer the question,
    just reformulate it if needed and otherwise return it as is."""

contextualized_qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", CONTEXTUALIZED_PROMPT),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{question}"),
        ]
    )

contextualized_qa_chain = contextualized_qa_prompt | llm | StrOutputParser()

chat_history = ChatMessageHistory()

query = "What are the prons and cons of the Proxy design pattern?"

ai_msg = contextualized_qa_chain.invoke(
    {
        'question': query, 
        'chat_history': chat_history.messages
    }
)
print(ai_msg)

chat_history.add_user_message(query)
chat_history.add_ai_message(ai_msg)

What are the advantages and disadvantages of using the Proxy design pattern?


In [43]:
def contextualized_question(input: dict):
        if input.get("chat_history"):
            return contextualized_qa_chain
        else:
            return input["question"]

qa_chain_with_memory = (
         RunnablePassthrough.assign(
            context=contextualized_question | qa_prompt | llm
        )
    )

query = "Can it be combined with other patterns?"
result = qa_chain_with_memory.invoke(
    {
        'question': query,  
        'chat_history': chat_history.messages
    }
)

display(Markdown(result['context'].content))

Yes, it is absolutely possible to combine the Proxy design pattern with other design patterns. In fact, doing so can enhance the functionality and flexibility of your system. Here are a few examples of how the Proxy pattern can be integrated with other design patterns:

1. **Decorator Pattern**: The Proxy pattern can be combined with the Decorator pattern to add additional responsibilities to an object dynamically. For instance, you could use a Proxy to control access to a resource while also using a Decorator to add logging or caching functionality.

2. **Singleton Pattern**: If you want to ensure that only one instance of a resource is accessed through a Proxy, you can combine it with the Singleton pattern. This way, the Proxy can manage access to the single instance of the resource, ensuring that it is only created once.

3. **Observer Pattern**: A Proxy can act as an intermediary that notifies observers about changes in the subject it is proxying. This can be useful in scenarios where you want to monitor access or changes to a resource.

4. **Factory Pattern**: You can use a Factory to create Proxy objects. This can be particularly useful if you want to create different types of Proxies (e.g., virtual, remote, or protection proxies) based on certain conditions.

5. **Command Pattern**: If the Proxy is used to control access to a command, you can combine it with the Command pattern to encapsulate requests as objects. The Proxy can then manage the execution of these commands, adding additional logic such as logging or access control.

6. **Strategy Pattern**: You can use a Proxy to switch between different strategies at runtime. For example, a Proxy could decide which strategy to use based on the current state of the application or user permissions.

7. **Adapter Pattern**: If you need to adapt an interface to a different one, you can use a Proxy as an Adapter. This allows you to control access to the adapted object while also providing a different interface.

When combining design patterns, it's essential to ensure that the resulting architecture remains clear and maintainable. Each pattern should serve a specific purpose, and the interactions between them should be well-defined to avoid unnecessary complexity.

In [44]:
result

{'question': 'Can it be combined with other patterns?',
 'chat_history': [HumanMessage(content='What are the prons and cons of the Proxy design pattern?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='What are the advantages and disadvantages of using the Proxy design pattern?', additional_kwargs={}, response_metadata={})],
 'context': AIMessage(content="Yes, it is absolutely possible to combine the Proxy design pattern with other design patterns. In fact, doing so can enhance the functionality and flexibility of your system. Here are a few examples of how the Proxy pattern can be integrated with other design patterns:\n\n1. **Decorator Pattern**: The Proxy pattern can be combined with the Decorator pattern to add additional responsibilities to an object dynamically. For instance, you could use a Proxy to control access to a resource while also using a Decorator to add logging or caching functionality.\n\n2. **Singleton Pattern**: If you want to ensure that only one

In [ ]:
# TODO: Add dynamic few shots

In [71]:
zero_shot_prompt = PromptTemplate(
    input_variables=['input'],
    template="""Return the antonym of the input given along with an explanation.
    Input: {input}
    Output:
    Explanation:
    """
)

# Zero-shot
zero_shot_chain = zero_shot_prompt | llm

query = 'happy' # 'I am very sad but still have hope'
result = zero_shot_chain.invoke(input=query)
print(result.content)

Output: sad  
Explanation: The antonym of "happy" is "sad." While "happy" describes a state of joy, contentment, or pleasure, "sad" refers to a state of unhappiness, sorrow, or disappointment. These two words represent opposite emotional states.


In [65]:
# Examples of a pretend task of creating antonyms.
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [75]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # The list of examples available to select from.
    examples,
    # The embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    Chroma, # The database to store the examples with their embeddings
    # The number of examples to produce.
    k=1,
)

similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Return the antonym of the input given along with an explanation. \n\nExamples:",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

print(similar_prompt.format(adjective="rainy"))


Return the antonym of the input given along with an explanation. 

Examples:

Input: sunny
Output: gloomy

Input: rainy
Output:


In [77]:
# Few-shot
few_shot_chain = similar_prompt | llm

query = 'rainy' # 'I am very sad but still have hope'
print(similar_prompt.format(adjective=query))
print()
result = few_shot_chain.invoke(input=query)
print(result.content)

Return the antonym of the input given along with an explanation. 

Examples:

Input: sunny
Output: gloomy

Input: rainy
Output:

Output: dry

Explanation: "Rainy" refers to weather characterized by rain, which implies moisture and wet conditions. The antonym "dry" describes a lack of moisture, indicating clear or arid conditions, which contrasts with the idea of rain.


In [82]:
class FormattedAntonym(BaseModel):
    antonym: str = Field(description="An antonym for the input word or phrase.")
    explanation: str = Field(description="A short explanation of how the antonym was generated")

llm_with_structure = llm.with_structured_output(FormattedAntonym)

few_shot_chain1 = similar_prompt | llm_with_structure
result = few_shot_chain1.invoke(input=query)
result

FormattedAntonym(antonym='dry', explanation="The antonym 'dry' contrasts with 'rainy' as it describes weather conditions that lack moisture, while 'rainy' indicates the presence of rain.")

In [85]:
# result.model_dump_json()
result.model_dump()

{'antonym': 'dry',
 'explanation': "The antonym 'dry' contrasts with 'rainy' as it describes weather conditions that lack moisture, while 'rainy' indicates the presence of rain."}